<a href="https://colab.research.google.com/github/qiyangsun/BDHA/blob/main/data_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder

### Loading data

In [23]:
# Step 1: Load data from a CSV file
file_path = "/content/drive/MyDrive/Colab Notebooks/analytical_ready_1124.csv"  # Replace with your CSV file path
df = pd.read_csv(file_path)
# Display the first few rows to understand the data structure
print("Original Data:")
print(df.head())

Original Data:
           unique_id          charttime_r  temperature_r  heartrate_r  \
0  10002755_31312310  2137-03-07 17:42:00           97.5        102.0   
1  10002755_31312310  2137-03-07 19:54:00           98.0         91.0   
2  10002755_31312310  2137-03-07 23:06:00           98.1        100.0   
3  10010920_34816072  2150-10-05 02:20:00           99.0         95.0   
4  10010920_34816072  2150-10-05 06:12:00           98.6         87.0   

   resprate_r  o2sat_r  sbp_r  dbp_r gender         race_r  ...     sbp_7  \
0        18.0      NaN  131.0   71.0      F  UNKNOWN_OTHER  ...  6.028144   
1        16.0     98.0  138.0   63.0      F  UNKNOWN_OTHER  ...  7.005081   
2        18.0     98.0  132.0   78.0      F  UNKNOWN_OTHER  ...  7.029545   
3        16.0     99.0  130.0   89.0      M          WHITE  ...  5.879107   
4        16.0     97.0  137.0   86.0      M          WHITE  ...  6.900827   

      sbp_8     dbp_1     dbp_2     dbp_3     dbp_4     dbp_5     dbp_6  \
0  5.817

### Encoding

In [24]:
columns_to_encode = ['gender', 'race_r']  # Replace with actual column names

# Initialize LabelEncoder
encoder = LabelEncoder()

# Apply Label Encoding to each column
for col in columns_to_encode:
    df[f'{col}_encoded'] = encoder.fit_transform(df[col])

df.drop(columns=columns_to_encode, inplace=True)

# Display the updated DataFrame
print("\nData After Encoding:")
print(df.head())


Data After Encoding:
           unique_id          charttime_r  temperature_r  heartrate_r  \
0  10002755_31312310  2137-03-07 17:42:00           97.5        102.0   
1  10002755_31312310  2137-03-07 19:54:00           98.0         91.0   
2  10002755_31312310  2137-03-07 23:06:00           98.1        100.0   
3  10010920_34816072  2150-10-05 02:20:00           99.0         95.0   
4  10010920_34816072  2150-10-05 06:12:00           98.6         87.0   

   resprate_r  o2sat_r  sbp_r  dbp_r  sepsis       time_admission  ...  \
0        18.0      NaN  131.0   71.0       0  2137-03-07 13:11:00  ...   
1        16.0     98.0  138.0   63.0       0  2137-03-07 13:11:00  ...   
2        18.0     98.0  132.0   78.0       0  2137-03-07 13:11:00  ...   
3        16.0     99.0  130.0   89.0       0  2150-10-04 23:37:00  ...   
4        16.0     97.0  137.0   86.0       0  2150-10-04 23:37:00  ...   

      dbp_1     dbp_2     dbp_3     dbp_4     dbp_5     dbp_6     dbp_7  \
0  5.837371  6.0026

### Drop extra

In [25]:
df.drop(columns=['time_admission','charttime_r','time_discharge'], inplace=True)
print(df.head())

           unique_id  temperature_r  heartrate_r  resprate_r  o2sat_r  sbp_r  \
0  10002755_31312310           97.5        102.0        18.0      NaN  131.0   
1  10002755_31312310           98.0         91.0        16.0     98.0  138.0   
2  10002755_31312310           98.1        100.0        18.0     98.0  132.0   
3  10010920_34816072           99.0         95.0        16.0     99.0  130.0   
4  10010920_34816072           98.6         87.0        16.0     97.0  137.0   

   dbp_r  sepsis  stay_length_hrs  timepoints  ...     dbp_1     dbp_2  \
0   71.0       0             9.95           1  ...  5.837371  6.002669   
1   63.0       0             9.95           2  ...  3.916302  4.062123   
2   78.0       0             9.95           3  ...  7.116859  7.010448   
3   89.0       0             8.55           1  ...  9.022676  9.006518   
4   86.0       0             8.55           2  ...  7.922969  8.134793   

      dbp_3     dbp_4     dbp_5     dbp_6     dbp_7     dbp_8  gender_enco

### Padding

In [26]:
### create the sequence
feature = [col for col in df.columns if col != 'unique_id']
sequences = df.groupby('unique_id')[feature].apply(lambda x: torch.tensor(x.values, dtype=torch.float32)).tolist()
# Step 3: Pad sequences to the same length
padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0.0)


In [31]:
padded_sequences.shape

torch.Size([4499, 109, 68])

['temperature_r',
 'heartrate_r',
 'resprate_r',
 'o2sat_r',
 'sbp_r',
 'dbp_r',
 'sepsis',
 'stay_length_hrs',
 'timepoints',
 'time_diff',
 'time_encoding_1',
 'time_encoding_2',
 'time_encoding_3',
 'time_encoding_4',
 'time_encoding_5',
 'time_encoding_6',
 'time_encoding_7',
 'time_encoding_8',
 'temperature_1',
 'temperature_2',
 'temperature_3',
 'temperature_4',
 'temperature_5',
 'temperature_6',
 'temperature_7',
 'temperature_8',
 'heartrate_1',
 'heartrate_2',
 'heartrate_3',
 'heartrate_4',
 'heartrate_5',
 'heartrate_6',
 'heartrate_7',
 'heartrate_8',
 'resprate_1',
 'resprate_2',
 'resprate_3',
 'resprate_4',
 'resprate_5',
 'resprate_6',
 'resprate_7',
 'resprate_8',
 'o2sat_1',
 'o2sat_2',
 'o2sat_3',
 'o2sat_4',
 'o2sat_5',
 'o2sat_6',
 'o2sat_7',
 'o2sat_8',
 'sbp_1',
 'sbp_2',
 'sbp_3',
 'sbp_4',
 'sbp_5',
 'sbp_6',
 'sbp_7',
 'sbp_8',
 'dbp_1',
 'dbp_2',
 'dbp_3',
 'dbp_4',
 'dbp_5',
 'dbp_6',
 'dbp_7',
 'dbp_8',
 'gender_encoded',
 'race_r_encoded']

### save

In [34]:
array_sequence = padded_sequences.numpy()

# Step 2: Flatten the sequence for CSV storage
# Each row in the CSV will represent one timestep (batch_index, sequence_index, feature_values)
flattened_data = []
for batch_idx, batch in enumerate(array_sequence):
    for time_idx, time_step in enumerate(batch):
        flattened_data.append([batch_idx, time_idx] + list(time_step))

# Step 3: Create a DataFrame
columns = ['batch_idx', 'time_idx'] + feature
df = pd.DataFrame(flattened_data, columns=columns)
df.drop(columns=['timepoints',''], inplace=True)
# Step 4: Save the DataFrame to a CSV file
output_file = "tensor_sequence.csv"
df.to_csv(output_file, index=False)
print(f"Tensor sequence saved to {output_file}")

Tensor sequence saved to tensor_sequence.csv
